matminer: https://hackingmaterials.lbl.gov/matminer/

scikit-learn: https://scikit-learn.org/

The Material Project Workshop: https://workshop.materialsproject.org/lessons/08_ml_matminer/matminer-notes/

In [225]:
!pip install folium==0.2.1
!pip install imgaug==0.2.5
!pip install jupyter-client==6.1.5
!pip install numpy==1.19.2
!pip install pandas
#!pip install -U pandas-profiling
!pip install pymatgen==2021.3.9
!pip install matminer==0.6.5
#!pip install figrecipes

  Using cached numpy-1.19.2-cp37-cp37m-manylinux2010_x86_64.whl (14.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.4
    Uninstalling numpy-1.21.4:
      Successfully uninstalled numpy-1.21.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymatgen 2021.3.9 requires numpy>=1.20.1, but you have numpy 1.19.2 which is incompatible.


  Using cached numpy-1.21.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.21.4 which is incompatible.


In [226]:
# You can get various training data by using Matminer.
# Import the function for getting the dataset 
from matminer.datasets import get_available_datasets

In [227]:
# View available data names and details 
get_available_datasets()

boltztrap_mp: Effective mass and thermoelectric properties of 8924 compounds in The  Materials Project database that are calculated by the BoltzTraP software package run on the GGA-PBE or GGA+U density functional theory calculation results. The properties are reported at the temperature of 300 Kelvin and the carrier concentration of 1e18 1/cm3.

brgoch_superhard_training: 2574 materials used for training regressors that predict shear and bulk modulus.

castelli_perovskites: 18,928 perovskites generated with ABX combinatorics, calculating gllbsc band gap and pbe structure, and also reporting absolute band edge positions and heat of formation.

citrine_thermal_conductivity: Thermal conductivity of 872 compounds measured experimentally and retrieved from Citrine database from various references. The reported values are measured at various temperatures of which 295 are at room temperature.

dielectric_constant: 1,056 structures with dielectric properties, calculated with DFPT-PBE.

double_

['boltztrap_mp',
 'brgoch_superhard_training',
 'castelli_perovskites',
 'citrine_thermal_conductivity',
 'dielectric_constant',
 'double_perovskites_gap',
 'double_perovskites_gap_lumo',
 'elastic_tensor_2015',
 'expt_formation_enthalpy',
 'expt_gap',
 'flla',
 'glass_binary',
 'glass_binary_v2',
 'glass_ternary_hipt',
 'glass_ternary_landolt',
 'heusler_magnetic',
 'jarvis_dft_2d',
 'jarvis_dft_3d',
 'jarvis_ml_dft_training',
 'm2ax',
 'matbench_dielectric',
 'matbench_expt_gap',
 'matbench_expt_is_metal',
 'matbench_glass',
 'matbench_jdft2d',
 'matbench_log_gvrh',
 'matbench_log_kvrh',
 'matbench_mp_e_form',
 'matbench_mp_gap',
 'matbench_mp_is_metal',
 'matbench_perovskites',
 'matbench_phonons',
 'matbench_steels',
 'mp_all_20181018',
 'mp_nostruct_20181018',
 'phonon_dielectric_mp',
 'piezoelectric_tensor',
 'steel_strength',
 'wolverton_oxides']

In [228]:
# Import a function to read a dataset 
from matminer.datasets import load_dataset

In [229]:
# Read the dataset Import the function to read the dataset "dielectric_constant" into the pandas data frame 
df = load_dataset("dielectric_constant")

In [230]:
# Display the beginning of the data frame 
df.head()

,material_id,formula,nsites,space_group,volume,structure,band_gap,e_electronic,e_total,n,poly_electronic,poly_total,pot_ferroelectric,cif,meta,poscar
0,mp-441,Rb2Te,3,225,159.501208,"[[1.75725875 1.2425695 3.04366125] Rb, [5.271...",1.88,"[[3.44115795, -3.097e-05, -6.276e-05], [-2.837...","[[6.23414745, -0.00035252, -9.796e-05], [-0.00...",1.86,3.44,6.23,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Rb2 Te1\n1.0\n5.271776 0.000000 3.043661\n1.75...
1,mp-22881,CdCl2,3,166,84.298097,"[[0. 0. 0.] Cd, [ 4.27210959 2.64061969 13.13...",3.52,"[[3.34688382, -0.04498543, -0.22379197], [-0.0...","[[7.97018673, -0.29423886, -1.463590159999999]...",1.78,3.16,6.73,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Cd1 Cl2\n1.0\n3.850977 0.072671 5.494462\n1.78...
2,mp-28013,MnI2,3,164,108.335875,"[[0. 0. 0.] Mn, [-2.07904300e-06 2.40067320e+...",1.17,"[[5.5430849, -5.28e-06, -2.5030000000000003e-0...","[[13.80606079, 0.0006911900000000001, 9.655e-0...",2.23,4.97,10.64,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Mn1 I2\n1.0\n4.158086 0.000000 0.000000\n-2.07...
3,mp-567290,LaN,4,186,88.162562,[[-1.73309900e-06 2.38611186e+00 5.95256328e...,1.12,"[[7.09316738, 7.99e-06, -0.0003864700000000000...","[[16.79535386, 8.199999999999997e-07, -0.00948...",2.65,7.04,17.99,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,La2 N2\n1.0\n4.132865 0.000000 0.000000\n-2.06...
4,mp-560902,MnF2,6,136,82.826401,"[[1.677294 2.484476 2.484476] Mn, [0. 0. 0.] M...",2.87,"[[2.4239622, 7.452000000000001e-05, 6.06100000...","[[6.44055613, 0.0020446600000000002, 0.0013203...",1.53,2.35,7.12,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLDAUTYPE ...,Mn2 F4\n1.0\n3.354588 0.000000 0.000000\n0.000...


In [231]:
df.tail()

,material_id,formula,nsites,space_group,volume,structure,band_gap,e_electronic,e_total,n,poly_electronic,poly_total,pot_ferroelectric,cif,meta,poscar
1051,mp-568032,Cd(InSe2)2,7,111,212.493121,"[[0. 0. 0.] Cd, [2.9560375 0. 3.03973 ...",0.87,"[[7.74896783, 0.0, 0.0], [0.0, 7.74896783, 0.0...","[[11.85159471, 1e-08, 0.0], [1e-08, 11.8515962...",2.77,7.67,11.76,True,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Cd1 In2 Se4\n1.0\n5.912075 0.000000 0.000000\n...
1052,mp-696944,LaHBr2,8,194,220.041363,"[[2.068917 3.58317965 3.70992025] La, [4.400...",3.60,"[[4.40504391, 6.1e-07, 0.0], [6.1e-07, 4.40501...","[[8.77136355, 1.649999999999999e-06, 0.0], [1....",2.00,3.99,7.08,True,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,La2 H2 Br4\n1.0\n4.137833 0.000000 0.000000\n-...
1053,mp-16238,Li2AgSb,4,216,73.882306,"[[1.35965225 0.96141925 2.354987 ] Li, [2.719...",0.14,"[[212.60750153, -1.843e-05, 0.0], [-1.843e-05,...","[[232.59707383, -0.0005407400000000001, 0.0025...",14.58,212.61,232.60,True,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Li2 Ag1 Sb1\n1.0\n4.078957 0.000000 2.354987\n...
1054,mp-4405,Rb3AuO,5,221,177.269065,"[[0. 2.808758 2.808758] Rb, [2.808758 2....",0.21,"[[6.40511712, 0.0, 0.0], [0.0, 6.40511712, 0.0...","[[22.43799785, 0.0, 0.0], [0.0, 22.4380185, 0....",2.53,6.41,22.44,True,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Rb3 Au1 O1\n1.0\n5.617516 0.000000 0.000000\n0...
1055,mp-3486,KSnSb,6,186,227.725015,[[-1.89006800e-06 2.56736395e+00 1.32914373e...,0.26,"[[13.85634957, 1.8e-06, 0.0], [1.8e-06, 13.856...","[[16.45311887, 1.64e-06, -0.00019139], [1.64e-...",3.53,12.47,15.55,True,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,K2 Sn2 Sb2\n1.0\n4.446803 0.000000 0.000000\n-...


In [232]:
# Calculate the basic statistics for each column in the data frame
# (Only columns of numerical data for which statistics can be calculated are displayed)
df["band_gap"]

0       1.88
1       3.52
2       1.17
3       1.12
4       2.87
        ... 
1051    0.87
1052    3.60
1053    0.14
1054    0.21
1055    0.26
Name: band_gap, Length: 1056, dtype: float64

In [233]:
# Fetch the 3rd row data (Note: python index starts from 0) 
df.iloc[2]

material_id                                                   mp-28013
formula                                                           MnI2
nsites                                                               3
space_group                                                        164
volume                                                         108.336
structure            [[0. 0. 0.] Mn, [-2.07904300e-06  2.40067320e+...
band_gap                                                          1.17
e_electronic         [[5.5430849, -5.28e-06, -2.5030000000000003e-0...
e_total              [[13.80606079, 0.0006911900000000001, 9.655e-0...
n                                                                 2.23
poly_electronic                                                   4.97
poly_total                                                       10.64
pot_ferroelectric                                                False
cif                  #\#CIF1.1\n###################################...
meta  

In [234]:
# Extract only row data whose "volume" is 580 or more
# (Refer to Boolean index: Extract only index whose condition is True) 
mask = (df["volume"] >= 580)
df[mask]

,material_id,formula,nsites,space_group,volume,structure,band_gap,e_electronic,e_total,n,poly_electronic,poly_total,pot_ferroelectric,cif,meta,poscar
206,mp-23280,AsCl3,16,19,582.085309,"[[0.13113333 7.14863883 9.63476955] As, [2.457...",3.99,"[[2.2839161900000002, 0.00014519, -2.238000000...","[[2.49739759, 0.00069379, 0.00075864], [0.0004...",1.57,2.47,3.30,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,As4 Cl12\n1.0\n4.652758 0.000000 0.000000\n0.0...
216,mp-9064,RbTe,12,189,590.136085,"[[6.61780282 0. 0. ] Rb, [1.750...",0.43,"[[3.25648277, 5.9650000000000007e-05, 1.57e-06...","[[5.34517928, 0.00022474000000000002, -0.00018...",2.05,4.20,6.77,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Rb6 Te6\n1.0\n10.118717 0.000000 0.000000\n-5....
219,mp-23230,PCl3,16,62,590.637274,"[[6.02561815 8.74038483 7.55586375] P, [2.7640...",4.03,"[[2.39067769, 0.00017593, 8.931000000000001e-0...","[[2.80467218, 0.00034093000000000003, 0.000692...",1.52,2.31,2.76,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,P4 Cl12\n1.0\n6.523152 0.000000 0.000000\n0.00...
251,mp-2160,Sb2Se3,20,62,597.341134,"[[3.02245275 0.42059268 1.7670481 ] Sb, [ 1.00...",0.76,"[[19.1521058, 5.5e-06, 0.00025268], [-1.078000...","[[81.93819038000001, 0.0006755800000000001, 0....",3.97,15.76,63.53,True,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Sb8 Se12\n1.0\n4.029937 0.000000 0.000000\n0.0...


In [235]:
# Extract only row data whose "band_gap" is greater than 0 and generate a new data frame 
mask = (df["band_gap"] > 0)
semiconductor_df = df[mask]
semiconductor_df

,material_id,formula,nsites,space_group,volume,structure,band_gap,e_electronic,e_total,n,poly_electronic,poly_total,pot_ferroelectric,cif,meta,poscar
0,mp-441,Rb2Te,3,225,159.501208,"[[1.75725875 1.2425695 3.04366125] Rb, [5.271...",1.88,"[[3.44115795, -3.097e-05, -6.276e-05], [-2.837...","[[6.23414745, -0.00035252, -9.796e-05], [-0.00...",1.86,3.44,6.23,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Rb2 Te1\n1.0\n5.271776 0.000000 3.043661\n1.75...
1,mp-22881,CdCl2,3,166,84.298097,"[[0. 0. 0.] Cd, [ 4.27210959 2.64061969 13.13...",3.52,"[[3.34688382, -0.04498543, -0.22379197], [-0.0...","[[7.97018673, -0.29423886, -1.463590159999999]...",1.78,3.16,6.73,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Cd1 Cl2\n1.0\n3.850977 0.072671 5.494462\n1.78...
2,mp-28013,MnI2,3,164,108.335875,"[[0. 0. 0.] Mn, [-2.07904300e-06 2.40067320e+...",1.17,"[[5.5430849, -5.28e-06, -2.5030000000000003e-0...","[[13.80606079, 0.0006911900000000001, 9.655e-0...",2.23,4.97,10.64,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Mn1 I2\n1.0\n4.158086 0.000000 0.000000\n-2.07...
3,mp-567290,LaN,4,186,88.162562,[[-1.73309900e-06 2.38611186e+00 5.95256328e...,1.12,"[[7.09316738, 7.99e-06, -0.0003864700000000000...","[[16.79535386, 8.199999999999997e-07, -0.00948...",2.65,7.04,17.99,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,La2 N2\n1.0\n4.132865 0.000000 0.000000\n-2.06...
4,mp-560902,MnF2,6,136,82.826401,"[[1.677294 2.484476 2.484476] Mn, [0. 0. 0.] M...",2.87,"[[2.4239622, 7.452000000000001e-05, 6.06100000...","[[6.44055613, 0.0020446600000000002, 0.0013203...",1.53,2.35,7.12,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLDAUTYPE ...,Mn2 F4\n1.0\n3.354588 0.000000 0.000000\n0.000...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,mp-568032,Cd(InSe2)2,7,111,212.493121,"[[0. 0. 0.] Cd, [2.9560375 0. 3.03973 ...",0.87,"[[7.74896783, 0.0, 0.0], [0.0, 7.74896783, 0.0...","[[11.85159471, 1e-08, 0.0], [1e-08, 11.8515962...",2.77,7.67,11.76,True,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Cd1 In2 Se4\n1.0\n5.912075 0.000000 0.000000\n...
1052,mp-696944,LaHBr2,8,194,220.041363,"[[2.068917 3.58317965 3.70992025] La, [4.400...",3.60,"[[4.40504391, 6.1e-07, 0.0], [6.1e-07, 4.40501...","[[8.77136355, 1.649999999999999e-06, 0.0], [1....",2.00,3.99,7.08,True,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,La2 H2 Br4\n1.0\n4.137833 0.000000 0.000000\n-...
1053,mp-16238,Li2AgSb,4,216,73.882306,"[[1.35965225 0.96141925 2.354987 ] Li, [2.719...",0.14,"[[212.60750153, -1.843e-05, 0.0], [-1.843e-05,...","[[232.59707383, -0.0005407400000000001, 0.0025...",14.58,212.61,232.60,True,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Li2 Ag1 Sb1\n1.0\n4.078957 0.000000 2.354987\n...
1054,mp-4405,Rb3AuO,5,221,177.269065,"[[0. 2.808758 2.808758] Rb, [2.808758 2....",0.21,"[[6.40511712, 0.0, 0.0], [0.0, 6.40511712, 0.0...","[[22.43799785, 0.0, 0.0], [0.0, 22.4380185, 0....",2.53,6.41,22.44,True,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Rb3 Au1 O1\n1.0\n5.617516 0.000000 0.000000\n0...


In [236]:
# Drop the four columns (["nsites", "space_group", "e_electronic", "e_total"])
# (When the axis number is 0, the row is deleted, and when it is 1, the column is deleted)
# Note: After deleting, you need to assign it to a variable in the dataframe (the original variable df has not changed) 
cleaned_df = df.drop(["nsites", "space_group", "e_electronic", "e_total"], axis=1)

In [237]:
cleaned_df.head()

,material_id,formula,volume,structure,band_gap,n,poly_electronic,poly_total,pot_ferroelectric,cif,meta,poscar
0,mp-441,Rb2Te,159.501208,"[[1.75725875 1.2425695 3.04366125] Rb, [5.271...",1.88,1.86,3.44,6.23,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Rb2 Te1\n1.0\n5.271776 0.000000 3.043661\n1.75...
1,mp-22881,CdCl2,84.298097,"[[0. 0. 0.] Cd, [ 4.27210959 2.64061969 13.13...",3.52,1.78,3.16,6.73,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Cd1 Cl2\n1.0\n3.850977 0.072671 5.494462\n1.78...
2,mp-28013,MnI2,108.335875,"[[0. 0. 0.] Mn, [-2.07904300e-06 2.40067320e+...",1.17,2.23,4.97,10.64,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Mn1 I2\n1.0\n4.158086 0.000000 0.000000\n-2.07...
3,mp-567290,LaN,88.162562,[[-1.73309900e-06 2.38611186e+00 5.95256328e...,1.12,2.65,7.04,17.99,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,La2 N2\n1.0\n4.132865 0.000000 0.000000\n-2.06...
4,mp-560902,MnF2,82.826401,"[[1.677294 2.484476 2.484476] Mn, [0. 0. 0.] M...",2.87,1.53,2.35,7.12,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLDAUTYPE ...,Mn2 F4\n1.0\n3.354588 0.000000 0.000000\n0.000...


Calculate the Ionic contribution from the total (static) dielectric constant ϵtotal and the dielectric constant ϵelectronic
$$
\epsilon_{\text{inonic}} = \epsilon_{\text{total}} - \epsilon_{\text{electronic}}
$$
If you specify a column and perform four arithmetic operations, it will calculate for each corresponding element.
And if the data frame does not have the specified column name, a new column will be created 

In [238]:
df["poly_ionic"] = df["poly_total"] - df["poly_electronic"]

In [239]:
# Confirm that the column has been added 
df.head()

,material_id,formula,nsites,space_group,volume,structure,band_gap,e_electronic,e_total,n,poly_electronic,poly_total,pot_ferroelectric,cif,meta,poscar,poly_ionic
0,mp-441,Rb2Te,3,225,159.501208,"[[1.75725875 1.2425695 3.04366125] Rb, [5.271...",1.88,"[[3.44115795, -3.097e-05, -6.276e-05], [-2.837...","[[6.23414745, -0.00035252, -9.796e-05], [-0.00...",1.86,3.44,6.23,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Rb2 Te1\n1.0\n5.271776 0.000000 3.043661\n1.75...,2.79
1,mp-22881,CdCl2,3,166,84.298097,"[[0. 0. 0.] Cd, [ 4.27210959 2.64061969 13.13...",3.52,"[[3.34688382, -0.04498543, -0.22379197], [-0.0...","[[7.97018673, -0.29423886, -1.463590159999999]...",1.78,3.16,6.73,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Cd1 Cl2\n1.0\n3.850977 0.072671 5.494462\n1.78...,3.57
2,mp-28013,MnI2,3,164,108.335875,"[[0. 0. 0.] Mn, [-2.07904300e-06 2.40067320e+...",1.17,"[[5.5430849, -5.28e-06, -2.5030000000000003e-0...","[[13.80606079, 0.0006911900000000001, 9.655e-0...",2.23,4.97,10.64,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,Mn1 I2\n1.0\n4.158086 0.000000 0.000000\n-2.07...,5.67
3,mp-567290,LaN,4,186,88.162562,[[-1.73309900e-06 2.38611186e+00 5.95256328e...,1.12,"[[7.09316738, 7.99e-06, -0.0003864700000000000...","[[16.79535386, 8.199999999999997e-07, -0.00948...",2.65,7.04,17.99,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLWAVE = F...,La2 N2\n1.0\n4.132865 0.000000 0.000000\n-2.06...,10.95
4,mp-560902,MnF2,6,136,82.826401,"[[1.677294 2.484476 2.484476] Mn, [0. 0. 0.] M...",2.87,"[[2.4239622, 7.452000000000001e-05, 6.06100000...","[[6.44055613, 0.0020446600000000002, 0.0013203...",1.53,2.35,7.12,False,#\#CIF1.1\n###################################...,{u'incar': u'NELM = 100\nIBRION = 8\nLDAUTYPE ...,Mn2 F4\n1.0\n3.354588 0.000000 0.000000\n0.000...,4.77


Descriptor generation for machine learning 

Learn how to generate descriptors for machine learning models.
matminer's featuresizers provide methods to generate descriptors from composition and structure (atomic arrangement).
See the official documentation for a list of descriptors that can be generated:
https://hackingmaterials.lbl.gov/matminer/featurizer_summary.html

Learn how to generate a descriptor from the pymatgen class Composition. First, create a Composition object with composition Fe2O3. 

In [240]:
from pymatgen import Composition

fe2o3 = Composition("Fe2O3")

In [241]:
# Import of class ElementProperty to generate elemental feature statistic as descriptor 
from matminer.featurizers.composition import ElementProperty

In [242]:
# List of names of elemental features
# The following class Element Parameters can be used: https://pymatgen.org/pymatgen.core.periodic_table.html 
prop_name = ["atomic_radius_calculated","molar_volume", "boiling_point", "melting_point","liquid_range"]

In [243]:
# Create an object of class ElementProperty
# Arguments: database name, elemental features, statistic (here weighted average) 
ep = ElementProperty("pymatgen",prop_name,['mean'])

In [244]:
# The name of the descriptor to be generated 
element_prop_labels = ep.feature_labels()
print(element_prop_labels)

['PymatgenData mean atomic_radius_calculated', 'PymatgenData mean molar_volume', 'PymatgenData mean boiling_point', 'PymatgenData mean melting_point', 'PymatgenData mean liquid_range']


In [245]:
element_weight_averaged_prop = ep.featurize(fe2o3)
print(element_weight_averaged_prop)

[0.9120000000000001, 13.251999999999999, 1307.72, 757.28, 550.4399999999999]


In [246]:
# Import of class ElementFraction to generate element ratio as descriptor 
from matminer.featurizers.composition import ElementFraction

In [247]:
# Create an object of class ElementFraction 
ef = ElementFraction()

In [248]:
# The name of the descriptor to be generated 
element_fraction_labels = ef.feature_labels()
print(element_fraction_labels)

['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr']


In [249]:
# Generate descriptor 
element_fractions = ef.featurize(fe2o3)
print(element_fractions)

[0, 0, 0, 0, 0, 0, 0, 0.6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [250]:
# Check the descriptor name and descriptor value together 
print(element_fraction_labels[7], element_fractions[7])
print(element_fraction_labels[25], element_fractions[25])

O 0.6
Fe 0.4


In [251]:
# In the above steps, you learned how to generate a descriptor for a single Composition object,
# When there are many compositions, it is tedious to generate descriptors one by one.
# In the following, you will learn the procedure for collectively generating the descriptors of the composition and structure contained in the data frame.

# Read the dataset contained in matminer 
from matminer.datasets.dataset_retrieval import load_dataset

df = load_dataset("brgoch_superhard_training")
df.head()

,formula,bulk_modulus,shear_modulus,composition,material_id,structure,brgoch_feats,suspect_value
0,AlPt3,225.230461,91.197748,"(Al, Pt)",mp-188,"[[0. 0. 0.] Al, [0. 1.96140395 1.96140...","{'atomic_number_feat_1': 123.5, 'atomic_number...",False
1,Mn2Nb,232.696340,74.590157,"(Mn, Nb)",mp-12659,[[-2.23765223e-08 1.42974191e+00 5.92614104e...,"{'atomic_number_feat_1': 45.5, 'atomic_number_...",False
2,HfO2,204.573433,98.564374,"(Hf, O)",mp-352,"[[2.24450185 3.85793022 4.83390736] O, [2.7788...","{'atomic_number_feat_1': 44.0, 'atomic_number_...",False
3,Cu3Pt,159.312640,51.778816,"(Cu, Pt)",mp-12086,"[[0. 1.86144248 1.86144248] Cu, [1.861...","{'atomic_number_feat_1': 82.5, 'atomic_number_...",False
4,Mg3Pt,69.637565,27.588765,"(Mg, Pt)",mp-18707,"[[0. 0. 2.73626461] Mg, [0. ...","{'atomic_number_feat_1': 57.0, 'atomic_number_...",False


In [252]:
# The column "composition" contains the Composition object for pymatgen.
# Only the element name is displayed, but confirm that there is also information such as composition ratio. 
print(df["composition"][2])
print(type(df["composition"][2]))

Hf1 O2
<class 'pymatgen.core.composition.Composition'>


In [253]:
# By the method featurerize_dataframe of the object of class ElementFraction
# Can generate descriptors for data frames. (1st argument: data frame, 2nd argument: composition)
# (Use package multiprocessing to calculate descriptors for each composition in parallel)

# Note: Must be reassigned to a variable in the data frame 
df = ef.featurize_dataframe(df, "composition")

ElementFraction:   0%|          | 0/2574 [00:00<?, ?it/s]

In [254]:
# Make sure the descriptor has been added 
df.head()

,formula,bulk_modulus,shear_modulus,composition,material_id,structure,brgoch_feats,suspect_value,H,He,Li,Be,B,C,N,O,F,Ne,Na,Mg,Al,Si,P,S,Cl,Ar,K,Ca,Sc,Ti,V,Cr,Mn,Fe,Co,Ni,Cu,Zn,Ga,Ge,...,Gd,Tb,Dy,Ho,Er,Tm,Yb,Lu,Hf,Ta,W,Re,Os,Ir,Pt,Au,Hg,Tl,Pb,Bi,Po,At,Rn,Fr,Ra,Ac,Th,Pa,U,Np,Pu,Am,Cm,Bk,Cf,Es,Fm,Md,No,Lr
0,AlPt3,225.230461,91.197748,"(Al, Pt)",mp-188,"[[0. 0. 0.] Al, [0. 1.96140395 1.96140...","{'atomic_number_feat_1': 123.5, 'atomic_number...",False,0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,0.0,0.00,0.25,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Mn2Nb,232.696340,74.590157,"(Mn, Nb)",mp-12659,[[-2.23765223e-08 1.42974191e+00 5.92614104e...,"{'atomic_number_feat_1': 45.5, 'atomic_number_...",False,0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HfO2,204.573433,98.564374,"(Hf, O)",mp-352,"[[2.24450185 3.85793022 4.83390736] O, [2.7788...","{'atomic_number_feat_1': 44.0, 'atomic_number_...",False,0,0,0.0,0.0,0.0,0.0,0.0,0.666667,0.0,0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.333333,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Cu3Pt,159.312640,51.778816,"(Cu, Pt)",mp-12086,"[[0. 1.86144248 1.86144248] Cu, [1.861...","{'atomic_number_feat_1': 82.5, 'atomic_number_...",False,0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.75,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Mg3Pt,69.637565,27.588765,"(Mg, Pt)",mp-18707,"[[0. 0. 2.73626461] Mg, [0. ...","{'atomic_number_feat_1': 57.0, 'atomic_number_...",False,0,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,0.0,0.75,0.00,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [255]:
# Next, learn how to generate descriptors from pymatgen's Structure object (crystal structure data).

# Load a new dataset containing Structure objects 
df = load_dataset("phonon_dielectric_mp")

df.head()

,mpid,eps_electronic,eps_total,last phdos peak,structure,formula
0,mp-1000,6.311555,12.773454,98.585771,"[[2.8943817 2.04663693 5.01321616] Te, [0. 0....",BaTe
1,mp-1002124,24.137743,32.965593,677.585725,"[[0. 0. 0.] Hf, [-3.78195772 -3.78195772 -3.78...",HfC
2,mp-1002164,8.111021,11.169464,761.585719,"[[0. 0. 0.] Ge, [ 3.45311592 3.45311592 -3.45...",GeC
3,mp-10044,10.032168,10.128936,701.585723,"[[0.98372595 0.69559929 1.70386332] B, [0. 0. ...",BAs
4,mp-1008223,3.979201,6.394043,204.585763,"[[0. 0. 0.] Ca, [ 4.95 4.95 -4.95] Se]",CaSe


In [256]:
# Make sure the column "structure" contains a Structure object 
print(df["structure"][0])

Full Formula (Ba1 Te1)
Reduced Formula: BaTe
abc   :   5.013216   5.013216   5.013216
angles:  60.000000  60.000000  60.000000
Sites (2)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Te    0.5  0.5  0.5
  1  Ba    0    0    0


In [257]:
# Import the class "DensityFeatures" that generates a descriptor about density 
from matminer.featurizers.structure import DensityFeatures

In [258]:
# Creating an object of class DensityFeatures 
densityf = DensityFeatures()

In [259]:
# Descriptor name 
densityf.feature_labels()

['density', 'vpa', 'packing fraction']

In [260]:
# Generate and add a density descriptor from the crystal structure information of the data frame 
df = densityf.featurize_dataframe(df, "structure")

DensityFeatures:   0%|          | 0/1296 [00:00<?, ?it/s]

In [261]:
# Confirm that 3 descriptors have been added 
df.head()

,mpid,eps_electronic,eps_total,last phdos peak,structure,formula,density,vpa,packing fraction
0,mp-1000,6.311555,12.773454,98.585771,"[[2.8943817 2.04663693 5.01321616] Te, [0. 0....",BaTe,4.937886,44.545547,0.596286
1,mp-1002124,24.137743,32.965593,677.585725,"[[0. 0. 0.] Hf, [-3.78195772 -3.78195772 -3.78...",HfC,9.868234,16.027886,0.531426
2,mp-1002164,8.111021,11.169464,761.585719,"[[0. 0. 0.] Ge, [ 3.45311592 3.45311592 -3.45...",GeC,5.760895,12.199996,0.394180
3,mp-10044,10.032168,10.128936,701.585723,"[[0.98372595 0.69559929 1.70386332] B, [0. 0. ...",BAs,5.087634,13.991016,0.319600
4,mp-1008223,3.979201,6.394043,204.585763,"[[0. 0. 0.] Ca, [ 4.95 4.95 -4.95] Se]",CaSe,2.750191,35.937000,0.428523


In [262]:
# matminer provides a class for converting pymatgen objects.
# The following example tries the class StrToComposition, which converts a string to a composition.

# Import class StrToComposition 
from matminer.featurizers.conversions import StrToComposition

In [263]:
# Create an object of class StrToComposition 
stc = StrToComposition()

In [264]:
# Display column "formula", check data class 
print(df["formula"][0])
print(type(df["formula"][0]))

BaTe
<class 'str'>


In [265]:
# Create a Composition object using the character string of the composition formula of the data frame 
df = stc.featurize_dataframe(df, "formula")

StrToComposition:   0%|          | 0/1296 [00:00<?, ?it/s]

In [266]:
# Confirm that Composition has been added 
df.head()

,mpid,eps_electronic,eps_total,last phdos peak,structure,formula,density,vpa,packing fraction,composition
0,mp-1000,6.311555,12.773454,98.585771,"[[2.8943817 2.04663693 5.01321616] Te, [0. 0....",BaTe,4.937886,44.545547,0.596286,"(Ba, Te)"
1,mp-1002124,24.137743,32.965593,677.585725,"[[0. 0. 0.] Hf, [-3.78195772 -3.78195772 -3.78...",HfC,9.868234,16.027886,0.531426,"(Hf, C)"
2,mp-1002164,8.111021,11.169464,761.585719,"[[0. 0. 0.] Ge, [ 3.45311592 3.45311592 -3.45...",GeC,5.760895,12.199996,0.394180,"(Ge, C)"
3,mp-10044,10.032168,10.128936,701.585723,"[[0.98372595 0.69559929 1.70386332] B, [0. 0. ...",BAs,5.087634,13.991016,0.319600,"(B, As)"
4,mp-1008223,3.979201,6.394043,204.585763,"[[0. 0. 0.] Ca, [ 4.95 4.95 -4.95] Se]",CaSe,2.750191,35.937000,0.428523,"(Ca, Se)"


In [267]:
# Make sure the column "composition" is an object of class Composition in pymatgen 
print(df["composition"][0])
print(type(df["composition"][0]))

Ba1 Te1
<class 'pymatgen.core.composition.Composition'>


Machine learning model training and prediction 

In [268]:
# Import function to load json format data 
from matminer.utils.io import load_dataframe_from_json

In [269]:
# Load the dataset for the elastic tensor of the material (data for the following papers)

# de Jong M, Chen W, Angsten T, Jain A, Notestine R, Gamst A, Sluiter M, Ande CK, van der Zwaag S,
# Plata JJ, Toher C, Curtarolo S, Ceder G, Persson KA, Asta M (2015)
# Charting the complete elastic properties of inorganic crystalline compounds. Scientific Data 2: 150009.
# https://doi.org/10.1038/sdata.2015.9
from google.colab import files
uploaded = files.upload() # elastic_tensor_2015_featurized.json
df = load_dataframe_from_json("elastic_tensor_2015_featurized.json")
df.head()

Saving elastic_tensor_2015_featurized.json to elastic_tensor_2015_featurized (2).json


,structure,formula,K_VRH,composition,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,MagpieData maximum MendeleevNumber,MagpieData range MendeleevNumber,MagpieData mean MendeleevNumber,MagpieData avg_dev MendeleevNumber,MagpieData mode MendeleevNumber,MagpieData minimum AtomicWeight,MagpieData maximum AtomicWeight,MagpieData range AtomicWeight,MagpieData mean AtomicWeight,MagpieData avg_dev AtomicWeight,MagpieData mode AtomicWeight,MagpieData minimum MeltingT,MagpieData maximum MeltingT,MagpieData range MeltingT,MagpieData mean MeltingT,MagpieData avg_dev MeltingT,MagpieData mode MeltingT,MagpieData minimum Column,MagpieData maximum Column,MagpieData range Column,MagpieData mean Column,MagpieData avg_dev Column,MagpieData mode Column,MagpieData minimum Row,MagpieData maximum Row,MagpieData range Row,MagpieData mean Row,MagpieData avg_dev Row,MagpieData mode Row,...,MagpieData mode NdUnfilled,MagpieData minimum NfUnfilled,MagpieData maximum NfUnfilled,MagpieData range NfUnfilled,MagpieData mean NfUnfilled,MagpieData avg_dev NfUnfilled,MagpieData mode NfUnfilled,MagpieData minimum NUnfilled,MagpieData maximum NUnfilled,MagpieData range NUnfilled,MagpieData mean NUnfilled,MagpieData avg_dev NUnfilled,MagpieData mode NUnfilled,MagpieData minimum GSvolume_pa,MagpieData maximum GSvolume_pa,MagpieData range GSvolume_pa,MagpieData mean GSvolume_pa,MagpieData avg_dev GSvolume_pa,MagpieData mode GSvolume_pa,MagpieData minimum GSbandgap,MagpieData maximum GSbandgap,MagpieData range GSbandgap,MagpieData mean GSbandgap,MagpieData avg_dev GSbandgap,MagpieData mode GSbandgap,MagpieData minimum GSmagmom,MagpieData maximum GSmagmom,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,density,vpa,packing fraction
0,"[[0.94814328 2.07280467 2.5112 ] Nb, [5.273...",Nb4CoSi,194.268884,"(Nb, Co, Si)",14.0,41.0,27.0,34.166667,9.111111,41.0,47.0,78.0,31.0,54.000000,9.333333,47.0,28.085500,92.906380,64.820880,76.440703,21.954237,92.90638,1687.00,2750.00,1063.00,2409.166667,454.444444,2750.00,5.0,14.0,9.0,7.166667,2.888889,5.0,3.0,5.0,2.0,4.500000,0.666667,5.0,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,7.0,4.0,5.833333,1.555556,7.0,10.2450,20.4400,10.195,17.234167,2.329722,18.1800,0.0,0.773,0.773,0.128833,0.214722,0.0,0.0,1.548471,1.548471,0.258079,0.430131,0.0,194.0,229.0,35.0,222.833333,9.611111,229.0,7.834556,16.201654,0.688834
1,"[[0. 0. 0.] Al, [1.96639263 1.13529553 0.75278...",Al(CoSi)2,175.449907,"(Al, Co, Si)",13.0,27.0,14.0,19.000000,6.400000,14.0,58.0,78.0,20.0,69.000000,8.800000,58.0,26.981539,58.933195,31.951656,40.203786,14.983527,28.08550,933.47,1768.00,834.53,1568.694000,254.089600,1687.00,9.0,14.0,5.0,11.800000,2.240000,9.0,3.0,4.0,1.0,3.400000,0.480000,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,2.0,3.800000,0.640000,3.0,10.2450,20.4400,10.195,15.570000,4.260000,10.2450,0.0,0.773,0.773,0.309200,0.371040,0.0,0.0,1.548471,1.548471,0.619388,0.743266,0.0,194.0,227.0,33.0,213.400000,15.520000,194.0,5.384968,12.397466,0.644386
2,"[[1.480346 1.480346 1.480346] Si, [0. 0. 0.] Os]",SiOs,295.077545,"(Si, Os)",14.0,76.0,62.0,45.000000,31.000000,14.0,57.0,78.0,21.0,67.500000,10.500000,57.0,28.085500,190.230000,162.144500,109.157750,81.072250,28.08550,1687.00,3306.00,1619.00,2496.500000,809.500000,1687.00,8.0,14.0,6.0,11.000000,3.000000,8.0,3.0,6.0,3.0,4.500000,1.500000,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,4.000000,0.000000,4.0,14.0900,20.4400,6.350,17.265000,3.175000,14.0900,0.0,0.773,0.773,0.386500,0.386500,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,194.0,227.0,33.0,210.500000,16.500000,194.0,13.968635,12.976265,0.569426
3,"[[0. 1.09045794 0.84078375

In [270]:
# Let the objective variable to be predicted be "K_VRH". This value indicates bulk modulus.
# Specify the attribute values of the data frame, extract only the information of numpy.array and assign it to the variable y. 
y = df['K_VRH'].values

print(y)

[194.26888436 175.44990675 295.07754499 ...  89.41816126  99.3845653
  35.93865993]


In [271]:
# Generate a matrix X consisting of descriptors (features) to be input to the machine learning model.
# The matrix X is a matrix of the size of the number of data × the number of descriptors.

# This matrix is generated by removing other than the descriptor (numerical data, other than the objective variable). 
X = df.drop(["structure", "formula", "composition", "K_VRH"], axis=1)

In [272]:
# Display the name of the descriptor 
print("There are {} possible descriptors:".format(X.columns))
print(X.columns)

There are Index(['MagpieData minimum Number', 'MagpieData maximum Number',
       'MagpieData range Number', 'MagpieData mean Number',
       'MagpieData avg_dev Number', 'MagpieData mode Number',
       'MagpieData minimum MendeleevNumber',
       'MagpieData maximum MendeleevNumber',
       'MagpieData range MendeleevNumber', 'MagpieData mean MendeleevNumber',
       ...
       'MagpieData mode GSmagmom', 'MagpieData minimum SpaceGroupNumber',
       'MagpieData maximum SpaceGroupNumber',
       'MagpieData range SpaceGroupNumber', 'MagpieData mean SpaceGroupNumber',
       'MagpieData avg_dev SpaceGroupNumber',
       'MagpieData mode SpaceGroupNumber', 'density', 'vpa',
       'packing fraction'],
      dtype='object', length=135) possible descriptors:
Index(['MagpieData minimum Number', 'MagpieData maximum Number',
       'MagpieData range Number', 'MagpieData mean Number',
       'MagpieData avg_dev Number', 'MagpieData mode Number',
       'MagpieData minimum MendeleevNumber',
 

In [273]:
# Import the machine learning model class RandomForestRegressor for predicting real numbers 
from sklearn.ensemble import RandomForestRegressor

In [274]:
# Create an object of class RandomForestRegressor 
rf = RandomForestRegressor(n_estimators=100, random_state=1)

In [275]:
# Train the model using the training data (pair of feature matrix X and objective variable y) 
rf.fit(X, y)

RandomForestRegressor(random_state=1)

In [276]:
# Predict the value of the objective variable from the features.
# Here, the objective variable of the training data is predicted 
y_pred = rf.predict(X)

In [277]:
# Import of numerical calculation package numpy 
import numpy as np

In [278]:
# Importing a function to calculate the mean squared error (MSE) 
from sklearn.metrics import mean_squared_error

In [279]:
# Calculate the MSE between the predicted and true values of the objective variable for the training data 
mse = mean_squared_error(y, y_pred)
print('training RMSE = {:.3f} GPa'.format(np.sqrt(mse)))

training RMSE = 7.272 GPa


In [280]:
# Import a class to perform K-fold Cross Validation
# Cross-validation: One of the methods for estimating the test error of a prediction model 
from sklearn.model_selection import KFold

In [281]:
# Creating an object of class KFold
# k = 10, set to shuffle the data order 
kfold = KFold(n_splits=10, shuffle=True)

In [282]:
# Import functions to calculate cross-validation scores 
from sklearn.model_selection import cross_val_score

In [283]:
# Perform cross-validation
# Arguments: Machine learning model rf, training data input (feature) X, objective variable y, score to be calculated, cross-validation object
# Negative value of MSE is set as the score 
scores = cross_val_score(rf, X, y, scoring='neg_mean_squared_error', cv=kfold)


In [284]:
# Calculates the square root (RMSE: Root MSE) of the absolute value of the calculated score value and displays its average value
# (Since it is executed k times, there are k scores)
rmse_scores = [np.sqrt(abs(s)) for s in scores]
print('Mean RMSE: {:.3f}'.format(np.mean(rmse_scores)))

Mean RMSE: 18.750


In [285]:
# A function that calculates the predicted value of test data calculated in the process of cross-validation 
from sklearn.model_selection import cross_val_predict

In [286]:
# Predicted value of test data in cross-validation 
y_pred = cross_val_predict(rf, X, y, cv=kfold)

In [287]:
# new version, 1/Nov/2021
# Ref: https://colab.research.google.com/github/mickeykubo/analytics_book/blob/master/plotly2_1.ipynb#scrollTo=eYzTAuIRNrhH
import plotly
import plotly.express as px
import plotly.graph_objects as go
# For Google Colab. or Jupyter Lab.
import plotly.io as pio
pio.renderers.default = "colab"
plotly.__version__

'5.3.1'

In [288]:
# old version
# Visualization of forecast results
# Since the horizontal axis is the true value and the vertical axis is the predicted value, it is better to have a lot of data on the diagonal line. 

#pf = PlotlyFig(x_title='DFT (MP) bulk modulus (GPa)',
#               y_title='Predicted bulk modulus (GPa)',
#               mode='notebook')

#pf.xy(xy_pairs=[(y, y_pred), ([0, 400], [0, 400])], 
#      labels=df['formula'], 
#      modes=['markers', 'lines'],
#      lines=[{}, {'color': 'black', 'dash': 'dash'}], 
#      showlegends=False)

In [289]:
# new version, 1/Nov/2021
# Ref: https://colab.research.google.com/github/mickeykubo/analytics_book/blob/master/plotly2_1.ipynb#scrollTo=eYzTAuIRNrhH
trace1 = go.Scatter(
    x = y,
    y = y_pred,
    text=df['formula'],
    mode = 'markers',
    marker= dict(size= 14,
                #line= dict(width=1),  # for mode='markers + lines',
                color= "red",
                opacity= 0.3
                ),
    name='predict',
)

trace2 = go.Scatter(
    x = y,
    y = y,
    mode = 'lines',
    line = dict(color='royalblue', width=1, dash='dash'),
    # dash options include 'dash', 'dot', and 'dashdot'
    name='guide',
)

layout = go.Layout(
    title=' ',
    xaxis=dict(
        title='DFT (MP) bulk modulus (GPa)',
        range=([0, 400]), 
        showline=True,
        linewidth=2,
        linecolor='black',
        color='black',
        ticks='inside',
    ),
    yaxis=dict(
        title='Predicted bulk modulus (GPa)',
        range=([0, 400]), 
        showline=True,
        linewidth=2,
        linecolor='black',
        color='black',
        ticks='inside',
    ),
    plot_bgcolor='white',
    hovermode='closest',
)

data = [trace1, trace2]

#plotly.offline.iplot(data)
fig = go.Figure(data, layout)
fig.show()

In [290]:
# Importance of features calculated in Random Forest 
rf.feature_importances_

array([2.77737706e-04, 6.86497802e-04, 4.19014378e-04, 9.38306579e-04,
       6.27788172e-04, 8.11685429e-04, 5.85205797e-03, 4.13582985e-04,
       5.68896006e-03, 3.99784395e-03, 2.29068933e-03, 4.00079790e-03,
       2.80170565e-04, 1.12108238e-03, 5.04161260e-04, 7.21240737e-04,
       6.66258777e-04, 2.65253041e-04, 6.31863354e-02, 2.61334748e-02,
       1.69886830e-03, 5.43940586e-01, 3.79683746e-03, 1.84024489e-03,
       2.00724094e-02, 2.74160018e-04, 2.89023628e-04, 1.65750614e-03,
       1.70010289e-03, 8.44812934e-03, 4.07321100e-05, 4.20522484e-05,
       8.69871167e-05, 1.12538000e-03, 1.05324451e-03, 4.16679765e-05,
       3.96170134e-04, 1.48598578e-03, 6.16768132e-04, 3.24518244e-03,
       8.40211726e-04, 6.08207520e-04, 2.48464536e-03, 5.38387869e-04,
       9.44676379e-04, 1.17935220e-02, 1.54290424e-03, 3.42464373e-03,
       1.62873316e-04, 4.59487184e-05, 1.77379083e-04, 1.35098601e-03,
       3.88133058e-04, 3.33992917e-04, 2.17490708e-04, 3.12382175e-05,
      

In [291]:
# old version

# Visualize features sorted in descending order of importance
# 1st line: Importance of features
# 2nd line: Name of feature quantity
# 3rd: Data index in descending order of features
# 4th and subsequent lines: Visualize up to the top 5 

#importances = rf.feature_importances_
#included = X.columns.values
#indices = np.argsort(importances)[::-1]

#pf = PlotlyFig(y_title='Importance (%)',
#               title='Feature by importances',
#               mode='notebook')

#pf.bar(x=included[indices][0:5], y=importances[indices][0:5])

In [292]:
# new version, 1/Nov/2021
# Ref: https://workshop.materialsproject.org/lessons/08_ml_matminer/matminer-notes/

# Visualize features sorted in descending order of importance
# 1st line: Importance of features
# 2nd line: Name of feature quantity
# 3rd: Data index in descending order of features
# 4th and subsequent lines: Visualize up to the top 5 

importances = rf.feature_importances_
included = X.columns.values
indices = np.argsort(importances)[::-1]

fig_bar = px.bar(
    x=included[indices][0:5], 
    y=importances[indices][0:5], 
    title="Feature by importances",
    labels={"x": "Parameter", "y": "Importance (%)"}
)
fig_bar.show()